In [ ]:
# 1. Import Modules

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch, os

from numpy import dot
from numpy.linalg import norm
from tqdm import tqdm

In [ ]:
# 2. 학습, 평가 데이터 구분
def createFolder(dir):
    try:
        if not os.path.exists(dir):
            os.makedirs(dir)
    except OSError:
        print("파일을 생성하는 과정에서 오류 발생")

A = pd.read_parquet(r"..\\01_Dataset\\A-PAT_dedup.parquet")
B = pd.read_parquet(r"..\\01_Dataset\\B-PAT_dedup.parquet")

A_0 = A[A["label"] == 0]
A_1 = A[A["label"] == 1]

B_0 = B[B["label"] == 0]
B_1 = B[B["label"] == 1]

A_test_0 = A_0.sample(frac=0.2, random_state=2024)
A_study_0 = A_0.drop(A_test_0.index)
A_test_1 = A_1.sample(frac=0.2, random_state=2024)
A_study_1 = A_1.drop(A_test_1.index)
A_test = pd.concat([A_test_0, A_test_1], ignore_index=True)
A_study = pd.concat([A_study_0, A_study_1], ignore_index=True)

B_test_0 = B_0.sample(frac=0.2, random_state=2024)
B_study_0 = B_0.drop(B_test_0.index)
B_test_1 = B_1.sample(frac=0.2, random_state=2024)
B_study_1 = B_1.drop(B_test_1.index)
B_test = pd.concat([B_test_0, B_test_1], ignore_index=True)
B_study = pd.concat([B_study_0, B_study_1], ignore_index=True)

createFolder(r"..\\01_Dataset\\A")
A_test.to_parquet(r"..\\01_Dataset\\A\\test.parquet")
A_study.to_parquet(r"..\\01_Dataset\\A\\study.parquet")
createFolder(r"..\\01_Dataset\\B")
B_test.to_parquet(r"..\\01_Dataset\\B\\test.parquet")
B_study.to_parquet(r"..\\01_Dataset\\B\\study.parquet")

In [ ]:
# 3. 데이터 벡터화
A = pd.read_parquet(r"..\\01_Dataset\\A\\study.parquet")
B = pd.read_parquet(r"..\\01_Dataset\\B\\study.parquet")

def payload_to_vector(payloads : list) -> list:
    ans = []
    for payload in payloads:
        result = [0 for _ in range(256)]
        for i in range(0, len(payload), 2):
            result[int(payload[i:i+2], 16)] += 1
        ans.append(result)
    return ans

A["vector"] = payload_to_vector(A["payload"])
B["vector"] = payload_to_vector(B["payload"])

A.to_parquet(r"..\\01_Dataset\\A\\study.parquet")
B.to_parquet(r"..\\01_Dataset\\B\\study.parquet")

In [ ]:
# 4. 데이터 클러스터링
# A = pd.read_parquet(r"..\\01_Dataset\\A\\study.parquet")
# B = pd.read_parquet(r"..\\01_Dataset\\B\\study.parquet")

def str_to_list(s : str) -> list:
    return s.split(',')

def cosSim(x, y) -> float:
    x = [int(x) for x in x]; y = [int(y) for y in y]
    return dot(x, y)/(norm(x)*norm(y))

def clustering(dataset : pd.DataFrame, r : float) -> pd.DataFrame:
    dataset["clusters"] = [0 for _ in range(len(dataset.index))]
    idx = 1
    while True:
        if (len(dataset.loc[dataset["clusters"] == 0]) == 0): break
        pivot = dataset.loc[dataset["clusters"] == 0].iloc[0]
        pivot["clusters"] = idx
        for i in dataset[dataset["clusters"] == 0].index:
            data = dataset.iloc[i]
            if (cosSim(pivot["vector"], data["vector"]) >= r):
                data["clusters"] = idx
        idx += 1
    return dataset

A.iloc[0]["vector"][0]
# print(str_to_list(A.iloc[0]["vector"]))
print(clustering(A, 0.5))